# 어파인 변환
* 이동 변환 (translation, shift)
* 전단 변환 (shear)
* 크기 변환 (scale)
* 회전 변환 (rotation)
* 대칭 변환 (flip)

## 이동 변환(Translation Transformation)
* x' = x + a
* y' = y + b

![](translation.png)
![](translation_M.png) 
즉, [x' y'] = M * [x y 1]  
>(**M** : **cv2.warpAffine() 함수의 인자**. **무조건 2x3 실수 행렬**)

## (실습) 이동 변환
x축 방향으로 150, y축 방향으로 100만큼 이동

In [2]:
import cv2
import numpy as np

def affine_translation():
    src = cv2.imread('tekapo.bmp')
    
    M = np.array([[1, 0, 150], [0, 1, 100]]).astype(np.float32)
    
    dst = cv2.warpAffine(src, M, (0, 0)) # (0, 0) : 원본 크기 유지 (dst 크기 신경 x)
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
affine_translation()

![](translation_result.png)

## 전단 변환(Shear Transformation)
직사각형 영상을 한쪽 방향으로 밀어 평행사변형으로 변형 (층밀림 변환)

![](shear_x.png)
* x축 방향(가로 방향)으로 밀었을 때, y 좌표가 증가함에 따라 x값 바뀜 (*y값은 동일*)   
![](shear_y.png)
* y축 방향(세로 방향)으로 밀었을 때, x 좌표가 증가함에 따라 y값 바뀜 (*x값은 동일*)  

![](shear_M.png)
> **M은 무조건 2x3 크기**여야 하므로, 뒤에 [0 0]을 더하더라도 M에 포함시켜야 함!

## (실습) 전단 변환

In [3]:
def affine_shear():
    src = cv2.imread('tekapo.bmp')
    
    rows = src.shape[0]
    cols = src.shape[1]
    
    mx = 0.3 # x축 방향(가로 방향)으로 전단 변환, y값의 0.3배만큼씩 밀기
    M = np.array([[1, mx, 0], [0, 1, 0]]).astype(np.float32)
    
    # y값이 증가함에 따라, 아래로 내려갈수록 x축으로 더 많이 밀림
    dst = cv2.warpAffine(src, M, (int(cols + rows * mx), rows)) # cv는 (w, h) 순서!!
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
affine_shear()

![](shear_result.png)
> cf. ***세로 방향으로도 해볼 것!***

## 크기 변환 (Scale Transformation)
[크기 변환 2가지 방법]
* 1. 어파인 변환 행렬 M 생성 후, cv2.warpAffine() 함수를 이용한다.
* 2. **cv2.resize() 함수에 interpolation**을 지정한다. (크기를 줄이거나 늘리므로)
    * dst = cv2.resize(src, dsize, fx, fy, interpolation)
    > cf. dsize(dst size) = (round(rows * fx), round(cols * fy))

![](scale.png)
![](scale_M.png)
(이것보단 주로 2번 방법인 **resize() 함수로 크기 변환**)

* INTER_LINEAR : 연산 속도 빠르고, 화질도 충분히 좋음 (default)
* INTER_CUBIC, INTER_LANCZOS4 : INTER_LINEAR 보다 더 좋은 화질
* INTER_AREA : 영상 축소시 유리

## (실습) 크기 변환

In [5]:
def affine_scale():
    src = cv2.imread('rose.bmp')
    
    dst1 = cv2.resize(src, (0, 0), fx=4, fy=4, interpolation = cv2.INTER_NEAREST)
    dst2 = cv2.resize(src, (1920, 1280)) # 디폴트 : INTER_LINEAR
    dst3 = cv2.resize(src, (1920, 1280), interpolation = cv2.INTER_CUBIC)
    dst4 = cv2.resize(src, (1920, 1280), interpolation = cv2.INTER_LANCZOS4)
    
    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1[400:800, 500:900])
    cv2.imshow('dst2', dst2[400:800, 500:900])
    cv2.imshow('dst3', dst3[400:800, 500:900])
    cv2.imshow('dst4', dst4[400:800, 500:900])
    cv2.waitKey()
    cv2.destroyAllWindows()
    
affine_scale() 

# dst1 -> dst4로 갈수록 점점 화질 좋아짐

![](scale_result1.png)

## 회전 변환(Rotation Transformation)
원점을 기준으로 영상을 **반시계 방향**으로 **세타**만큼 회전하는 변환  


[회전 변환 2가지 방법]
* 1. 직접 **cos(), sin()** 으로 아래의 회전 변환 행렬 M 만들고 cv2.warpAffine()
* 2. **cv2.getRotationMatrix2D()** 로 M 만들고 cv2.warpAffine()  
> **cv2.getRotationMatrix2D()** : 원점뿐만 아니라 **특정 좌표 기준으로 회전** 가능, **크기 변환**까지도 함께 수행 가능한 어파인 변환 행렬 return
  
  
![](rotation.png)
> cf. 여기서 왜 cos **-sin sin** cos 가 아니라, cos **sin -sin** cos 일까?!

## (참고) 회전 변환 행렬 : "좌표계"의 차이!
* 일반 좌표계 : 회전각이 양수일 때 **반시계** 방향 (우상단이 1사분면)
* in **그래픽스** : 회전각이 양수일 때 **시계** 방향!
> 따라서, **반시계 방향으로 맞춰주기 위해서 "- 세타"** 로 처리!   

![](rotation_M.png)  
회전 변환 행렬 M을 구하는 2가지 방법
* **cos()와 sin()함수** 이용
* OpenCV에서 **cv2.getRotationMatrix2D()** 함수도 제공!  
(위처럼 cos()와 sin()를 이용해서 2x3짜리 M 행렬 만들어도 상관 x)  
> **cv2.getRotationMatrix2D(center, angle, scale)**

![](rotation2.png)
***수식 이해하기...!***

## (실습) 회전 변환
src의 **중심을 기준으로 반시계 방향으로 20도** 회전

In [6]:
def affine_rotation():
    src = cv2.imread('tekapo.bmp')
    
    cp = (src.shape[1] / 2, src.shape[0] / 2) # cp : center point
    M = cv2.getRotationMatrix2D(cp, 20, 1) # (center, angle, scale)
    
    dst = cv2.warpAffine(src, M, (0, 0))
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
affine_rotation()

![](rotation_result.png)

## 대칭 변환 (Flip Transformation)
**좌우** 대칭 변환
* x' = **w - 1 - x !!**
* y' = y
![](flip.png)  
![](flip_lr.png)  
  
**상하** 대칭 변환
* x' = x
* y' = **h - 1 - y !!**
![](flip_ud.png)  


* **cv2.flip(src, flipCode)** 이용
    * flipCode > 0 : 좌우 대칭
    * flipCode = 0 : 상하 대칭
    * flipCode < 0 : 상하, 좌우 대칭 모두 수행

## (실습) 대칭 변환

In [7]:
def affine_flip():
    src = cv2.imread('eastsea.bmp')
    cv2.imshow('src', src)
    
    for flip_code in [1, 0, -1]:
        dst = cv2.flip(src, flip_code)
        
        desc = 'flipCode : %d' % flip_code
        cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('dst', dst)
        cv2.waitKey()
        
    cv2.destroyAllWindows()
    
affine_flip()

![](flip_result.png)

<hr>

# 투시 변환 (Projective Transformation)
어파인 변환보다 **자유도가 높은** 영상의 기하학적 변환
* 직선의 평행 관계 유지 X
* 점 4개가 자유롭게 이동
* 점 하나의 이동 관계 ==> x좌표에 대한 방정식 & y좌표에 대한 방정식 (2개의 방정식)
    * 따라서, **총 8개의 방정식** 얻을 수 있음  

  
* Mp(투시변환 행렬) : 8개의 파라미터로 표현 가능. But, **계산의 편의상(여러 변환 행렬의 곱 미리 수행) 9개의 원소를 갖는 3x3 행렬** 사용  
![](perspective.png)

### cf. 동차 좌표계 (homogeneous coordinate)
입력 및 출력 좌표를 (x, y, 1), (wx', wy', w) 형태로 표현한 것 (**for. 좌표 계산의 편의!**)
![](pers_w.png)

* **cv2.getPerspectiveTransform()** : 투시 변환 행렬 M 구하는 함수

## (실습) 투시 변환